In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('all.csv')

D:\Pyplatform\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
df_cy = pd.read_csv('CalendarYear of each company.csv')

In [18]:
# dictionary of number of data points for each company
dfdict = {}
dfkf = df['KF_ID'].value_counts().sort_index().to_frame()
for j in range(len(dfkf)):
    if dfkf.index[j] not in dfdict.keys():
        dfdict[dfkf.index[j]] = int(dfkf.values[j])
    else:
        dfdict[dfkf.index[j]] += int(dfkf.values[j])

In [21]:
# dictionary of number of years we have information on each company
numberofyears = {}
for i in range(len(df_cy)):
    numberofyears[df_cy['KF_ID'][i]] = df_cy.iloc[i].drop('KF_ID').values.sum()

In [23]:
# dictionary of industry name of each company
isn = {}
for j in range(len(df)):
    if df['KF_ID'][j] not in isn.keys():
        isn[df['KF_ID'][j]] = [df['IndustryName'][j]]
    else:
        if df['IndustryName'][j] not in isn[df['KF_ID'][j]]:
            isn[df['KF_ID'][j]].append(df['IndustryName'][j])

In [25]:
# dictionary of industry segment name of each company
isegment = {}
df['IndustrySegmentName'].fillna('unknown',inplace = True)
for j in range(len(df)):
    if df['KF_ID'][j] not in isegment.keys():
        isegment[df['KF_ID'][j]] = [df['IndustrySegmentName'][j]]
    else:
        if df['IndustrySegmentName'][j] not in isegment[df['KF_ID'][j]]:
            isegment[df['KF_ID'][j]].append(df['IndustrySegmentName'][j])

In [160]:
# dictionary of job name of each company
# df = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20,df21,df22,df23,df24,df25,df26,df27,df28,df29]
# jobname = {}
# for i in df:
#     for j in range(len(i)):
#         if i['KF_ID'][j] not in jobname.keys():
#             jobname[i['KF_ID'][j]] = [i['JobName'][j]]
#         else:
#             if i['JobName'][j] not in jobname[i['KF_ID'][j]]:
#                 jobname[i['KF_ID'][j]].append(i['JobName'][j])

In [26]:
mydict = [dfdict,numberofyears,isn,isegment]
final_df = pd.DataFrame(mydict)
col = ['Number of Data Points for Each Company','Number of Years We Have Information','IndustryName','IndustrySegmentName']
final_data = pd.DataFrame(final_df.values.T,index = final_df.columns,columns = col)
final_data.to_csv('Company_Profiler.csv', encoding='gbk')

In [30]:
cp.head()

,KF_ID,Number of Data Points for Each Company,Number of Years We Have Information,IndustryName,IndustrySegmentName
0,1,8,1,['Financials'],['Consumer Finance']
1,2,14058,7,"['Consumer Services', 'Retail (Services)']","['Specialized Consumer Services', 'unknown']"
2,3,87477,10,"['Consumer Goods', 'Fast Moving Consumer Goods...","['Household and Personal Care', 'unknown']"
3,4,30390,10,"['Retailers (Services)', 'Consumer Services', ...","['unknown', 'Food Retailers and Wholesalers']"
4,5,77,2,['Industrials'],"['Transportation Infrastructure', 'Road and Ra..."


In [5]:
# Analyze positions of a company
def jobs_of_company(dataf,kf_id):
    jb = {}
    for i in range(len(dataf)):
        if dataf['KF_ID'][i] == kf_id:
            if dataf['KF_ID'][i] not in jb.keys():
                aa = {}
                aa[dataf['JobName'][i]] = [dataf['CalendarYear'][i]]
                jb[dataf['KF_ID'][i]] = aa
            else:
                if dataf['JobName'][i] not in jb[dataf['KF_ID'][i]].keys():
                    jb[dataf['KF_ID'][i]][dataf['JobName'][i]] = [dataf['CalendarYear'][i]]
                else:
                    if dataf['CalendarYear'][i] not in jb[dataf['KF_ID'][i]][dataf['JobName'][i]]:
                        jb[dataf['KF_ID'][i]][dataf['JobName'][i]].append(dataf['CalendarYear'][i])
        else:
            continue
    job_list = pd.DataFrame(jb)
    for j in job_list.values:
        j[0].sort()
    return job_list

In [6]:
job_list = jobs_of_company(df,4)

In [99]:
job_list.head()

,4
Accountant I,"[2016, 2018]"
Accountant I - 1008,[2010]
Accountant II,"[2012, 2013, 2014, 2016, 2018]"
Accountant II - 1007,"[2008, 2009, 2011]"
Accountant III,"[2012, 2013, 2014, 2016, 2018]"
